# Development notes for pyxelate

## Display ratio calculation

Round to quarters for ratios below 3, to halves for ratios below 5, and to integer numbers for all greater ratios.

In [1]:
import math
from fractions import Fraction

uproundable = Fraction(17, 10)
exact = Fraction(15, 10)

uprounded_ratio = Fraction(math.ceil(uproundable * 4), 4)
exact_ratio = Fraction(math.ceil(exact * 4), 4)

print('Uprounded %s: %s' % (uproundable, uprounded_ratio))
print('Exact %s: %s' % (exact, exact_ratio))

Uprounded 17/10: 7/4
Exact 3/2: 3/2


In [2]:
def display_ratio(image_size, canvas_size):
    """Display ratio calculated per dimension"""
    if image_size < canvas_size:
        return 1
    #
    raw_ratio = Fraction(image_size, canvas_size)
    if raw_ratio <= 3:
        return Fraction(math.ceil(raw_ratio * 4), 4)
    #
    if raw_ratio <= 5:
        return Fraction(math.ceil(raw_ratio * 2), 2)
    #
    return math.ceil(raw_ratio)

In [3]:
print(display_ratio(17, 10))
print(display_ratio(17, 5))
print(display_ratio(17, 3))

7/4
7/2
6


## pixelate-video considerations

### Add a ttk.progressbar in a modal window for long-running tasks

<https://tkdocs.com/shipman/ttk-Progressbar.html> with `mode='determinate'` if possible

Use cases:
- Splitting a video file into frames (after selecting a video file)
- Applying pixelations to the selected frames (apply and continue, save as…)
- Joining the frames to generate the result video (save as…)
- Adding the audio stream to the result video (save as…)

### Generic workflow

- Select a video file
- Create a temporary directory (cleanup an existing one before)
- Split the video into frames -> `{tempdir}/original` directory (see <https://github.com/Wikinaut/utils/wiki#How_to_extract_every_nth_frame>))
- Select the first frame for pixelation using a scale (<https://tkdocs.com/shipman/scale.html>, ending the last but one frame) and select the pixelation parameters like in pixelate_image
- Select the last frame for pixelation using a scale (starting at the frame directly following the first frame) and select pixelation parameters _except tilesize and shape_
- When clicking 'Apply and continue':
  - Apply the pixelations from start frame to end frame
  - Replace the matching originals with these frames
  - Start over at selecting the first frame
- When clicking 'Save as…':
  - Apply the pixelations from start frame to end frame
  - Join all frames into a new video, re-using all applicable information from the original video
  - Add the original audio stream(s) of the original video, if any
  - Cleanup the temporary directory
  - exit

### "Touched" flag

- Set to `False` after loading a video
- Set to `True` after first change on the "Select first frame" panel
- Set to `False` after saving

### Getting video information

via <https://gist.github.com/nrk/2286511>: `ffprobe -v quiet -print_format json -show_streams -show_format <video_file>`
        
Example output:
```
$ ffprobe -v quiet -print_format json -show_streams -show_format nosound-example.mp4 
{
    "streams": [
        {
            "index": 0,
            "codec_name": "h264",
            "codec_long_name": "H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10",
            "profile": "High 4:4:4 Predictive",
            "codec_type": "video",
            "codec_tag_string": "avc1",
            "codec_tag": "0x31637661",
            "width": 720,
            "height": 576,
            "coded_width": 720,
            "coded_height": 576,
            "closed_captions": 0,
            "has_b_frames": 2,
            "sample_aspect_ratio": "64:45",
            "display_aspect_ratio": "16:9",
            "pix_fmt": "yuv444p",
            "level": 30,
            "chroma_location": "left",
            "refs": 1,
            "is_avc": "true",
            "nal_length_size": "4",
            "r_frame_rate": "25/1",
            "avg_frame_rate": "25/1",
            "time_base": "1/12800",
            "start_pts": 0,
            "start_time": "0.000000",
            "duration_ts": 404992,
            "duration": "31.640000",
            "bit_rate": "3401977",
            "bits_per_raw_sample": "8",
            "nb_frames": "791",
            "disposition": {
                "default": 1,
                "dub": 0,
                "original": 0,
                "comment": 0,
                "lyrics": 0,
                "karaoke": 0,
                "forced": 0,
                "hearing_impaired": 0,
                "visual_impaired": 0,
                "clean_effects": 0,
                "attached_pic": 0,
                "timed_thumbnails": 0
            },
            "tags": {
                "language": "und",
                "handler_name": "VideoHandler",
                "vendor_id": "[0][0][0][0]"
            }
        }
    ],
    "format": {
        "filename": "nosound-example.mp4",
        "nb_streams": 1,
        "nb_programs": 0,
        "format_name": "mov,mp4,m4a,3gp,3g2,mj2",
        "format_long_name": "QuickTime / MOV",
        "start_time": "0.000000",
        "duration": "31.640000",
        "size": "13465185",
        "bit_rate": "3404597",
        "probe_score": 100,
        "tags": {
            "major_brand": "isom",
            "minor_version": "512",
            "compatible_brands": "isomiso2avc1mp41",
            "encoder": "Lavf58.20.100"
        }
    }
}
```

### ffmpeg video decomposition including progress output

see <https://github.com/Wikinaut/utils/wiki#How_to_extract_every_nth_frame> and <https://stackoverflow.com/a/43980180>: `ffmpeg -v quiet -i <video_file> -progress - <output_file_name_pattern>`

For example:
```
$ ffmpeg -v quiet -i nosound-example.mp4 -progress - frames/frame%04d.jpg
frame=1
fps=0.00
stream_0_0_q=4.3
bitrate=N/A
total_size=N/A
out_time_us=40000
out_time_ms=40000
out_time=00:00:00.040000
dup_frames=0
drop_frames=0
speed=0.841x
progress=continue
frame=44
fps=0.00
stream_0_0_q=24.8
bitrate=N/A
total_size=N/A
out_time_us=1760000
out_time_ms=1760000
out_time=00:00:01.760000
dup_frames=0
drop_frames=0
speed=3.19x
progress=continue
frame=90
fps=85.45
stream_0_0_q=24.8
bitrate=N/A
total_size=N/A
out_time_us=3600000
out_time_ms=3600000
out_time=00:00:03.600000
dup_frames=0
drop_frames=0
speed=3.42x
progress=continue

(...)

frame=693
fps=91.04
stream_0_0_q=24.8
bitrate=N/A
total_size=N/A
out_time_us=27720000
out_time_ms=27720000
out_time=00:00:27.720000
dup_frames=0
drop_frames=0
speed=3.64x
progress=continue
frame=740
fps=91.22
stream_0_0_q=24.8
bitrate=N/A
total_size=N/A
out_time_us=29600000
out_time_ms=29600000
out_time=00:00:29.600000
dup_frames=0
drop_frames=0
speed=3.65x
progress=continue
frame=788
fps=91.41
stream_0_0_q=24.8
bitrate=N/A
total_size=N/A
out_time_us=31520000
out_time_ms=31520000
out_time=00:00:31.520000
dup_frames=0
drop_frames=0
speed=3.66x
progress=continue
frame=791
fps=91.38
stream_0_0_q=24.8
bitrate=N/A
total_size=N/A
out_time_us=31640000
out_time_ms=31640000
out_time=00:00:31.640000
dup_frames=0
drop_frames=0
speed=3.66x
progress=end
```

### Combine Audio and video streams

see <https://stackoverflow.com/a/11783474>

The following should work:
```
ffmpeg -i <pixelated_video> -i <original_video> -map 0 -map 1:a -c copy <finished_video>
```

### Combining the pixelated frames to a video

see <https://trac.ffmpeg.org/wiki/Slideshow#Sequential> and <https://trac.ffmpeg.org/wiki/Encode/H.264>

use `-pix_fmt yuv420p` (<https://trac.ffmpeg.org/wiki/Slideshow#Colorspaceconversionandchromasub-sampling>)

### ffmpeg documentation

<https://ffmpeg.org/documentation.html>

### Encoding test

`ffmpeg -v quiet -progress - -framerate 25 -i frames/frame%04d.jpg -c:v libx264 -vf "fps=25,format=yuv420p" out.mp4`

-> succeeded, resulting in a slightly smaller output file than the original.

Maybe it yould be a good idea to provide the option to add parameters manually (some kind of "expert mode"). 


### Temporary directory

One for original frames, and one for mofified frames